# Distributed HPO with Ray Tune and XGBoost-Ray

This demo introduces **Ray tune's** key concepts using a a classification example. This example is derived from [Hyperparametere Tuning with Ray Tune and XGBoost-Ray](https://github.com/ray-project/xgboost_ray#hyperparameter-tuning). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use tune to execute your training, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#blendsearch) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine analyse the results
 
 <img src="https://docs.ray.io/en/latest/_images/tune-workflow.png" height="50%" width="60%">


See also the [Hyperparameter Tuning References](References-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [1]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

import ray
from ray import tune
CONNECT_TO_ANYSCALE=True

In [2]:
if ray.is_initialized:
    ray.shutdown()
    if CONNECT_TO_ANYSCALE:
        ray.init("anyscale://jsd-weekly-demo")
    else:
        ray.init()

Authenticating
Loaded Anyscale authentication token from ANYSCALE_CLI_TOKEN.

Output
(anyscale +2.9s) WARNING: No working_dir specified! Files will only be uploaded to the cluster if a working_dir is provided or a project is detected. In the future, files will only be uploaded if working_dir is provided. To ensure files continue being imported going forward, set the working_dir in your runtime environment. See https://docs.ray.io/en/latest/handling-dependencies.html#runtime-environments.
(anyscale +3.4s) .anyscale.yaml found in project_dir. Directory is attached to a project.
(anyscale +4.4s) Using project (name: prj-weekly-demo, project_dir: /Users/jules/git-repos/ray-core-tutorial, id: prj_5rvR1w2ciyUs9RM27FeZ6FVB).
(anyscale +7.6s) cluster jsd-weekly-demo is currently running, the cluster will not be restarted.


2022-01-05 16:17:49,561	INFO packaging.py:352 -- Creating a file package for local directory '/Users/jules/git-repos/ray-core-tutorial'.
2022-01-05 16:17:49,585	INFO packaging.py:221 -- Pushing file package 'gcs://_ray_pkg_bd137497e8c59fde.zip' (1.94MiB) to Ray cluster...
2022-01-05 16:17:52,156	INFO packaging.py:224 -- Successfully pushed file package 'gcs://_ray_pkg_bd137497e8c59fde.zip'.


(anyscale +34.6s) Connected to jsd-weekly-demo, see: https://console.anyscale.com/projects/prj_5rvR1w2ciyUs9RM27FeZ6FVB/clusters/ses_GKnU1MGrXqZNmAkicHxSKU9G
(anyscale +34.6s) URL for head node of cluster: https://session-gknu1mgrxqznmakichxsku9g.i.anyscaleuserdata.com


## Step 1: Define a 'Trainable' training function to use with Ray Tune `ray.tune(...)`

In [3]:
NUM_OF_ACTORS = 4           # degree of parallel trials; each actor will have a separate trial
NUM_OF_CPUS_PER_ACTOR = 1   # number of CPUs per actor

In [4]:
ray_params = RayParams(num_actors=NUM_OF_ACTORS, cpus_per_actor=NUM_OF_CPUS_PER_ACTOR)

In [5]:
def train_func_model(config:dict, checkpoint_dir=None):
    # create the dataset
    train_X, train_y = load_breast_cancer(return_X_y=True)
    # Convert to RayDMatrix data structure
    train_set = RayDMatrix(train_X, train_y)

    # Empty dictionary for the evaluation results reported back
    # to tune
    evals_result = {}

    # Train the model with XGBoost train
    bst = train(
        params=config,                       # our hyperparameter search space
        dtrain=train_set,                    # our RayDMatrix data structure
        evals_result=evals_result,           # place holder for results
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)                # distributed parameters configs for Ray Tune

    bst.save_model("model.xgb")

## Step 2: Define a hyperparameter search space

In [6]:
 # Specify the hyperparameter search space
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

## Step 3: Run Ray tune main trainer and examine the results

Ray Tune will launch distributed HPO, using four remote actors, each with its own instance of the trainable func

<img src="images/ray_tune_dist_hpo.png" height="50%" width="60%"> 

In [7]:
# Run tune
analysis = tune.run(
    train_func_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources()
)

(run pid=17661) == Status ==
(run pid=17661) Current time: 2022-01-05 16:18:05 (running for 00:00:00.12)
(run pid=17661) Memory usage on this node: 3.1/62.0 GiB
(run pid=17661) Using FIFO scheduling algorithm.
(run pid=17661) Resources requested: 0/80 CPUs, 0/0 GPUs, 0.0/216.2 GiB heap, 0.0/92.17 GiB objects
(run pid=17661) Result logdir: /home/ray/ray_results/train_func_model_2022-01-05_16-18-04
(run pid=17661) Number of trials: 4/4 (4 PENDING)
(run pid=17661) +------------------------------+----------+-------+-------------+-------------+-------------+
(run pid=17661) | Trial name                   | status   | loc   |         eta |   max_depth |   subsample |
(run pid=17661) |------------------------------+----------+-------+-------------+-------------+-------------|
(run pid=17661) | train_func_model_1d96d_00000 | PENDING  |       | 0.000913403 |           7 |    0.872278 |
(run pid=17661) | train_func_model_1d96d_00001 | PENDING  |       | 0.0319993   |           8 |    0.720519 |


(ImplicitFunc pid=17733) 2022-01-05 16:18:07,547	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=1626, ip=172.31.74.136) 2022-01-05 16:18:07,612	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=1655, ip=172.31.85.248) 2022-01-05 16:18:07,644	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=1358, ip=172.31.89.96) 2022-01-05 16:18:09,628	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=17733) 2022-01-05 16:18:09,665	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=17797) [16:18:09] task [xgboost.ray]:140110013325808 got new rank 3
(_RemoteRayXGBoostActor pid=17794) [16:18:09] task [xgboost.ray]:140011386044912 

(run pid=17661) Result for train_func_model_1d96d_00000:
(run pid=17661)   date: 2022-01-05_16-18-10
(run pid=17661)   done: false
(run pid=17661)   experiment_id: f60d9120bf7a414aa308e7a475ac1290
(run pid=17661)   hostname: ip-172-31-57-130
(run pid=17661)   iterations_since_restore: 1
(run pid=17661)   node_ip: 172.31.57.130
(run pid=17661)   pid: 17733
(run pid=17661)   time_since_restore: 3.1184630393981934
(run pid=17661)   time_this_iter_s: 3.1184630393981934
(run pid=17661)   time_total_s: 3.1184630393981934
(run pid=17661)   timestamp: 1641428290
(run pid=17661)   timesteps_since_restore: 0
(run pid=17661)   train-error: 0.029877
(run pid=17661)   train-logloss: 0.69234
(run pid=17661)   training_iteration: 1
(run pid=17661)   trial_id: 1d96d_00000
(run pid=17661)   
(run pid=17661) == Status ==
(run pid=17661) Current time: 2022-01-05 16:18:10 (running for 00:00:05.46)
(run pid=17661) Memory usage on this node: 3.7/62.0 GiB
(run pid=17661) Using FIFO scheduling algorithm.
(run

(ImplicitFunc pid=17733) 2022-01-05 16:18:10,606	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.11 seconds (0.94 pure XGBoost training time).


(run pid=17661) Authenticating


(ImplicitFunc pid=1358, ip=172.31.89.96) 2022-01-05 16:18:11,747	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=1480, ip=172.31.89.96) [16:18:11] task [xgboost.ray]:139885649756704 got new rank 1
(_RemoteRayXGBoostActor pid=1481, ip=172.31.89.96) [16:18:11] task [xgboost.ray]:139810916438560 got new rank 2
(_RemoteRayXGBoostActor pid=1479, ip=172.31.89.96) [16:18:11] task [xgboost.ray]:140519621620256 got new rank 0
(_RemoteRayXGBoostActor pid=1482, ip=172.31.89.96) [16:18:11] task [xgboost.ray]:139724400329248 got new rank 3


(run pid=17661) Loaded Anyscale authentication token from variable.
(run pid=17661) 
(run pid=17661) 2022-01-05 16:18:22,389	INFO command_runner.py:357 -- Fetched IP: 172.31.74.136
(run pid=17661) 2022-01-05 16:18:22,389	INFO log_timer.py:25 -- NodeUpdater: ins_bFQmuSwNVtjwmteVTfsQE6Qu: Got IP  [LogTimer=363ms]
(run pid=17661) == Status ==
(run pid=17661) Current time: 2022-01-05 16:18:23 (running for 00:00:18.08)
(run pid=17661) Memory usage on this node: 3.2/62.0 GiB
(run pid=17661) Using FIFO scheduling algorithm.
(run pid=17661) Resources requested: 15.0/80 CPUs, 0/0 GPUs, 0.0/216.2 GiB heap, 0.0/92.17 GiB objects
(run pid=17661) Current best trial: 1d96d_00000 with train-error=0.012302 and parameters={'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0009134032210727258, 'subsample': 0.8722780805626716, 'max_depth': 7, 'nthread': 1, 'n_jobs': 1}
(run pid=17661) Result logdir: /home/ray/ray_results/train_func_model_2022-01-05_16-

(ImplicitFunc pid=1655, ip=172.31.85.248) 2022-01-05 16:18:35,128	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 27.52 seconds (25.36 pure XGBoost training time).
(ImplicitFunc pid=1358, ip=172.31.89.96) 2022-01-05 16:18:35,129	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 25.74 seconds (23.38 pure XGBoost training time).
(ImplicitFunc pid=1626, ip=172.31.74.136) 2022-01-05 16:18:35,113	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 27.54 seconds (25.38 pure XGBoost training time).
(run pid=17661) 2022-01-05 16:18:35,255	INFO tune.py:626 -- Total run time: 30.99 seconds (30.04 seconds for the tuning loop).


In [11]:
print("Best hyperparameters", analysis.best_config)

Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0016991434240133134, 'subsample': 0.9118830368337653, 'max_depth': 7}


In [12]:
analysis.results_df.head(5)

,train-logloss,train-error,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,...,iterations_since_restore,experiment_tag,config.tree_method,config.objective,config.eval_metric,config.eta,config.subsample,config.max_depth,config.nthread,config.n_jobs
trial_id,,,,,,,,,,,,,,,,,,,,,
1d96d_00000,0.685199,0.012302,0.004223,True,None,None,10,f60d9120bf7a414aa308e7a475ac1290,2022-01-05_16-18-10,1641428290,...,10,"0_eta=0.0009134,max_depth=7,subsample=0.87228",approx,binary:logistic,"[logloss, error]",0.000913,0.872278,7,1,1
1d96d_00001,0.478342,0.019332,0.005906,True,None,None,10,58b79055036a4e79be3eb96d8352c38b,2022-01-05_16-18-35,1641428315,...,10,"1_eta=0.031999,max_depth=8,subsample=0.72052",approx,binary:logistic,"[logloss, error]",0.031999,0.720519,8,1,1
1d96d_00002,0.541761,0.057996,0.005068,True,None,None,10,b688b45bc1754c258ba1182c40c39bd8,2022-01-05_16-18-35,1641428315,...,10,"2_eta=0.02646,max_depth=1,subsample=0.9506",approx,binary:logistic,"[logloss, error]",0.026460,0.950598,1,1,1
1d96d_00003,0.678362,0.010545,0.005612,True,None,None,10,7fd2c0791a04419fa78e5ea26dc8b0ef,2022-01-05_16-18-35,1641428315,...,10,"3_eta=0.0016991,max_depth=7,subsample=0.91188",approx,binary:logistic,"[logloss, error]",0.001699,0.911883,7,1,1


In [13]:
ray.shutdown()

---

## References

 * [Ray Train: Tune: Scalable Hyperparameter Tuning](https://docs.ray.io/en/master/tune/index.html)
 * [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray)
 * [How to Speed Up XGBoost Model Training](https://www.anyscale.com/blog/how-to-speed-up-xgboost-model-training)
 * [XGBoost-Ray Project](https://github.com/ray-project/xgboost_ray)
 * [Distributed XGBoost on Ray](https://docs.ray.io/en/latest/xgboost-ray.html)